常见的错误处理方式有两类：一类是函数返回错误码，由调用方决定如何处理；另一类是以专用语句保护代码块，当异常发生时，跳转到指定的处理单元。前者有较好的性能，而后者可分离正常逻辑与故障处理代码。

将错误码设计为函数签名的一部分，可预先让调用方知道可能会发生什么，该机制无须处理调用堆栈信息，无须额外状态保存和流程控制机制，其简单、高效。但逻辑处理中充斥着冗长的错误处理代码会直接导致代码可读性变差，不便于维护和重构，结构化处理会更优雅一些。 

In [1]:
def test():
    try:
        raise Exception
    except:
        print('except')

In [2]:
import dis
dis.dis(test)

  2           0 SETUP_EXCEPT             8 (to 10)

  3           2 LOAD_GLOBAL              0 (Exception)
              4 RAISE_VARARGS            1
              6 POP_BLOCK
              8 JUMP_FORWARD            20 (to 30)

  4     >>   10 POP_TOP
             12 POP_TOP
             14 POP_TOP

  5          16 LOAD_GLOBAL              1 (print)
             18 LOAD_CONST               1 ('except')
             20 CALL_FUNCTION            1
             22 POP_TOP
             24 POP_EXCEPT
             26 JUMP_FORWARD             2 (to 30)
             28 END_FINALLY
        >>   30 LOAD_CONST               0 (None)
             32 RETURN_VALUE


解释器使用块栈(block stack)结构处理异常逻辑，它和执行栈一起被栈帧管理。  
0 向块栈添加except跳转位置  
2 创建并引发异常  
4 解释器从块栈弹出设置，按参数跳转  
10 跳转到此处  
12 清楚exc参数  
16 执行except代码  
24 移除块栈内的设置  
26 处理后函数正常返回

In [3]:
import sys
sys.exc_info()

(None, None, None)

In [11]:
import traceback
def b():
    raise Exception('sdsd')
def test():
    try:
        b()
    except:
        a = sys.exc_info()
        print(a)
        traceback.print_tb(a[2])

In [12]:
test()

(<class 'Exception'>, Exception('sdsd',), <traceback object at 0x110a8a088>)


  File "<ipython-input-11-0d385584402a>", line 6, in test
    b()
  File "<ipython-input-11-0d385584402a>", line 3, in b
    raise Exception('sdsd')


异常对象保存在当前线程状态里，异常发生时，解释器会逐级为每个未能捕获异常的函数调用创建关联跟踪对象traceback，它负责存储栈帧、最后指令位置以及源码行等信息。异常一旦被捕获处理，保存在线程状态内的exc_type, exc_value, exc_traceback都会被清除

异常处理结构4个组成单元：
- try: 需要保护的代码块
- except: 异常发生时，按所属类型捕获并处理
- else: 异常未发生时执行，但最少有一个except语句
- finally: 无论异常是否发生，总是执行

即便在try语句块内有return、break等逻辑跳转指令，解释器也会确保finally被执行。  
建议以Exception、Error后缀区分可修复异常和不可修复错误。

In [13]:
class UserException(Exception):
    def __init__(self, uid):
        self.uid = uid  # 用于用户修复逻辑
        super().__init__(f'an exception about {self.uid}', uid) # 用于日志和显示

In [14]:
import logging
def test():
    try:
        raise UserException(103)
    except UserException as exc:
        logging.error(exc.args)  # 日志记录
        print(exc.uid)  # 修复或重新抛出

In [15]:
test()

ERROR:root:('an exception about 103', 103)


103


日志和显示信息使用Exception.args，有助于通用性，但若用于诊断和修复数据，建议使用显式属性存储。

warnings模块继承自Exception，但默认只输出相关信息，不会做为异常抛出，但可配置忽略(-Wi)，或转换为异常抛出(-We)。